# Question 1, 2

In [1]:
#imports
import nltk
from sklearn.model_selection import train_test_split

# custom functions
from helper import *

D:\Users\apoor\Miniconda3\envs\nlp\lib\site-packages\sklearn\utils\__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\apoor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
filename = "../books/adventures_sherlock_holmes.txt"
sentences = getSents(filename)

In [3]:
# lower case words, and remove punctuations.
normalizedSentences = lowerPunct(sentences)

# add Start stop symbols
finalSentences = addStartStop(normalizedSentences)

# divide the corpus into test and train
test, train = train_test_split(sentences,\
                               test_size=0.2,\
                               random_state=42)

----

# Question 3

In [4]:
# DEMO
se = [['<s>', 'i', 'am', 'not',  'dog', 'food', '</s>'],
      ['<s>', 'i', 'love', 'dog',  'food', 'yummy', '</s>'],
      ['<s>', 'dog', 'is', 'not', 'food', '</s>']]

# get the counts of the required nGrams

print ("2gram counts:")
nGramCount(se, 2)

2gram counts:


{'<s> i': 2,
 'i am': 1,
 'am not': 1,
 'not dog': 1,
 'dog food': 2,
 'food </s>': 2,
 'i love': 1,
 'love dog': 1,
 'food yummy': 1,
 'yummy </s>': 1,
 '<s> dog': 1,
 'dog is': 1,
 'is not': 1,
 'not food': 1}

In [5]:
k = 3
MLE(nGramCount(se, k), nGramCount(se, k-1))

{'<s> i am': 0.5,
 'i am not': 1.0,
 'am not dog': 1.0,
 'not dog food': 1.0,
 'dog food </s>': 0.5,
 '<s> i love': 0.5,
 'i love dog': 1.0,
 'love dog food': 1.0,
 'dog food yummy': 0.5,
 'food yummy </s>': 1.0,
 '<s> dog is': 1.0,
 'dog is not': 1.0,
 'is not food': 1.0,
 'not food </s>': 1.0}

-----

In [6]:
# finding the MLEs for all the ngrams
N = 4
mles = []
nGrams = [None]
for i in range(1, N + 1):
    nGrams.append(nGramCount(finalSentences, i))
    
for i in range(1, N + 1):
    mles.append(MLE(nGrams[i], nGrams[i-1]))

In [7]:
# change the value of ngram to get it's MLE
ngram = 3
mles[ngram - 1]

{'<s> adventures of': 1.0,
 'adventures of sherlock': 0.5,
 'of sherlock holmes': 1.0,
 'sherlock holmes by': 0.020618556701030927,
 'holmes by sir': 0.5,
 'by sir arthur': 1.0,
 'sir arthur conan': 1.0,
 'arthur conan doyle': 1.0,
 'conan doyle i': 1.0,
 'doyle i </s>': 1.0,
 '<s> a scandal': 0.020833333333333332,
 'a scandal in': 0.75,
 'scandal in bohemia': 0.6666666666666666,
 'in bohemia ii': 0.25,
 'bohemia ii </s>': 1.0,
 '<s> the league': 0.0058309037900874635,
 'the league iii': 0.06666666666666667,
 'league iii </s>': 1.0,
 '<s> a case': 0.020833333333333332,
 'a case of': 0.5555555555555556,
 'case of identity': 0.125,
 'of identity iv': 0.5,
 'identity iv </s>': 1.0,
 '<s> the boscombe': 0.011661807580174927,
 'the boscombe valley': 0.36363636363636365,
 'boscombe valley mystery': 0.3333333333333333,
 'valley mystery the': 0.5,
 'mystery the five': 1.0,
 'the five orange': 0.5,
 'five orange pips': 1.0,
 'orange pips vi': 0.14285714285714285,
 'pips vi </s>': 1.0,
 '<s> the

In [8]:
# Possible number of ngrams possible
poss_avai=[]
for i in range(1, N+1):
    poss_avai.append(possible_avail(nGrams[i]))

In [9]:
# first val contains the possible no of ith ngrams
# second val is the number of unique ith ngrams
# where i > 1
s = ["unigrams", "bigrams", "trigrams", "quadgrams"]
for so in range(len(s)):
    print (s[so])
    print ("possible:", poss_avai[so][0])
    print ("present:", str(poss_avai[so][1]) + '\n')

unigrams
possible: 28959855
present: 7611

bigrams
possible: 1142491701
present: 47802

trigrams
possible: 3436329351
present: 82902

quadgrams
possible: 4166338686
present: 91284



----

# Question 4
### part a

In [11]:
for i in range(N, 0, -1):
    print (Generator(mles[i-1]))

<s> really holmes said merryweather </s>
<s> lestrade laughed </s>
<s> my orders </s>
<s> </s>
